# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

El siguiente trabajo consiste en el desarrollo de un bot que conteste las preguntas de los clientes de una empresa de servicios. En este caso, se ha escogido la empresa Chilexpress. El bot 'Eva' ha sido entrenado de dos formas distintas, con un embedding (como Word2Vec o Elmo) y mediante un transformer.

# 1. Creación y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

La compañía escogida para realizar este trabajo es Chilexpress. Su sección de preguntas y respuestas se muestra en el siguiente enlace:

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

La tabla 'tablaQA.csv' consiste en una columna con pregunta y otra con respuestas. Como se observa en esta sección, la tabla consiste en 46 preguntas y sus respectivas respuestas.  

In [4]:
!wget https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/tablaQA.csv -O /content/tablaQA.csv

--2021-12-31 00:28:02--  https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/tablaQA.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14591 (14K) [text/plain]
Saving to: ‘/content/tablaQA.csv’

/content/tablaQA.cs 100%[===================>]  14.25K  --.-KB/s    in 0s      

2021-12-31 00:28:02 (52.9 MB/s) - ‘/content/tablaQA.csv’ saved [14591/14591]



In [7]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep='\t')
qa.head()

,Pregunta,Respuesta
0,¿Qué es la Orden de Transporte (OT)?,La Orden de Transporte (OT) es el número asoci...
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."


In [ ]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

La siguiente tabla corresponde a las preguntas de la sección QA de Chilexpress con la clase "información" y se han inventado los ejemplos de las clases "saludo", "despedida" y "nombre".

In [5]:
!wget https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/tiposmensajes.csv -O /content/tiposmensajes.csv

--2021-12-31 00:28:06--  https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/tiposmensajes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7505 (7.3K) [text/plain]
Saving to: ‘/content/tiposmensajes.csv’

/content/tiposmensa 100%[===================>]   7.33K  --.-KB/s    in 0s      

2021-12-31 00:28:06 (67.9 MB/s) - ‘/content/tiposmensajes.csv’ saved [7505/7505]



In [8]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=';')
messages.head()

,Clase,Mensaje
0,saludo,Hola
1,saludo,Buenas noches
2,saludo,Buenos días
3,saludo,Buen día
4,saludo,Buen Lunes


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

Se generan respuestas por defecto para cada clase que hemos inventado, de esta forma, Eva puede responder a una pregunta de esta clase con una de las respuestas de esta lista. 

In [ ]:
!wget https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/respuestasDefecto.csv -O /content/respuestasDefecto.csv

--2021-12-31 00:00:35--  https://raw.githubusercontent.com/alvaro-neira/nlp-homework2/main/respuestasDefecto.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2281 (2.2K) [text/plain]
Saving to: ‘/content/respuestasDefecto.csv’

/content/respuestas 100%[===================>]   2.23K  --.-KB/s    in 0s      

2021-12-31 00:00:35 (47.8 MB/s) - ‘/content/respuestasDefecto.csv’ saved [2281/2281]



In [ ]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=';')
answers.head()

,Clase,Mensaje
0,saludo,"Hola, ¿en qué puedo ayudarte?"
1,saludo,"Hola, ¿en qué puedo ayudarlo?"
2,saludo,"Hola, cuéntame qué necesitas"
3,saludo,"Hola, cuénteme qué necesita"
4,saludo,"Hola, cuéntame en qué puedo ayudarte"


## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva.



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

El embedding utilizado para codificar el texto de entrada es Wor2Vec. Para ello, primero se preprocesa con un normalizador y un tokenizador. 

In [ ]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  #if len(tokens)==1:
  #  tokens=text
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [ ]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))


In [ ]:
word2vec_model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed, min_count=1)

## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

Se define la función para predecir la clase del texto de entrada. 

### 2.2.1 Word2Vec

Se ajusta el modelo con los mensajes preprocesados. 

In [ ]:
features = np.zeros(shape=(len(messages), word2vec_model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg), word2vec_model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

0.5010510510510511

In [ ]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens, word2vec_model).reshape(1, -1)
  return clf.predict(vector)

In [ ]:
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(features,np.array(messages.Clase))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

Ejemplo de las preguntas y las clases obtenidas para cada una de ellas con el modelo. 

In [ ]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
print(predict_class('hola buenas'))
print(predict_class('hola'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))
print(predict_class('hasta luego'))
print(predict_class('adios'))

['información']
['nombre']
['saludo']
['saludo']
['información']
['información']
['despedida']
['despedida']


## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

Primero, se preprocesa el texto de la misma forma que en el punto anterior, pero se agrega la eliminación de stopwords. 

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pattern = r"""(?x)                   # set flag to allow verbose regexps
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A.
              |\$?\d+(?:[.,]\d+)?%?  # numbers, incl. currency and percentages
              |\w+(?:[-']\w+)*       # words w/ optional internal hyphens/apostrophe
              |(?:[+/\-@&*¡!.,])     # special characters with meanings
            """

def normalizer(text, remove_tildes = True): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = nltk.regexp_tokenize(text, pattern)
    text = ' '.join(text)
    if remove_tildes:
        text = re.sub('á', 'a', text) #replaces special vowels to their base forms
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [ ]:
nltk_stopwords = nltk.corpus.stopwords.words('spanish')
stopwords_normalized = [normalizer(word) for word in nltk_stopwords]

def no_stopwords(processed):
  '''
  Función para eliminar las stopwords de un texto
  '''
  result = []
  for w in processed:
    if w not in stopwords_normalized:
      result.append(w)

  return result

In [ ]:
corpus_question = qa.Pregunta.tolist()
questions_processed = []
for question in corpus_question:
  pre = preprocessor(question)
  pre = no_stopwords(pre)
  questions_processed.append(' '.join(pre))

Ejemplo de las preguntas preprocesadas.

In [ ]:
questions_processed[:5]

['orden transporte ot',
 'plazo retirar envio sucursal chilexpress',
 'puede persona retirar envio sucursal chilexpress',
 'puede modificar direccion envio',
 'puedo cambiar destino envio direccion particular sucursal chilexpress']

En esta sección se realiza una modificación del modelo Word2Vec para utilizar Doc2Vec para identificar la pregunta en la base de datos que sea más similar a la que se ha preguntado.

Referencias: 
* https://www.linkedin.com/pulse/how-find-semantic-similar-sentences-from-your-dataset-jayaraman/
* https://radimrehurek.com/gensim/models/doc2vec.html

In [ ]:
# <1 min
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(questions_processed)]

max_epochs = 2000

doc2_model = Doc2Vec(
    vector_size=150,
    alpha=0.001, 
    min_alpha=0.002,
    dm=1,
    window=2,
    min_count=1,
    workers=4
)
doc2_model.build_vocab(tagged_data)

# Train the model based on epochs parameter
for epoch in range(max_epochs):
    doc2_model.train(tagged_data,
                total_examples=doc2_model.corpus_count,
                epochs=doc2_model.epochs)

# Save model. 
doc2_model.save("similar_sentence.doc2_model")

Se crea una función get_answer que entrega la respuesta asociada a la predicción con el modelo doc2vec. Recibe tres parámetros:
* question: la pregunta que se espera obtener la respuesta. (requerida)
* topn: top n mejores resultados encontrados. (opcional)
* debug: Verdadero para imprimir más detalles de los resultados. (opcional)

In [ ]:
doc2_model= Doc2Vec.load("similar_sentence.doc2_model")

def get_answer(question, topn=1, debug=False):
  tokens = preprocessor(question)
  tokens = no_stopwords(tokens)
  vector = doc2_model.infer_vector(tokens)
  most_similar = doc2_model.docvecs.most_similar([vector], topn=topn)
  list_answers = qa.Respuesta.tolist()
  top = 1
  for index,value in most_similar:
    log = """
    --Top {}------------------------------
      pregunta similar    : {}
      respuesta asosciada : {}
      valor similitud     : {}
    --------------------------------------
    """.format(
        top,
        corpus_question[int(index)],
        list_answers[int(index)],
        value
    )
    if debug:
      print(log)
    else:
      print(list_answers[int(index)])
    top += 1


Algunos ejemplos de las preguntas realizadas y las 3 primeras identificaciones en la base de datos (pregunta-respuesta-valor). 

In [ ]:
get_answer('¿Qué es la Orden de Transporte (OT)?', 3, True)


    --Top 1------------------------------
      pregunta similar    : ¿Qué pasa si el destinatario no puede recibir el envío en su dirección?
      respuesta asosciada : Chilexpress realiza dos intentos de entrega en el domicilio, en dos días hábiles y horarios distintos, los que son notificados al destinatario mediante un email. Si no se puede efectuar la entrega, el envío será devuelto al remitente.
      valor similitud     : 0.574269711971283
    --------------------------------------
    

    --Top 2------------------------------
      pregunta similar    : ¿Se puede modificar la dirección de envío?
      respuesta asosciada : Sí, el cambio puede ser solicitado por el remitente siempre y cuando lo haga el mismo día que contrató el servicio y la nueva dirección corresponda a la misma comuna del despacho original. Para realizar el cambio de dirección debes llamar a nuestro Call Center 600 2000 102. En el caso de una compra online, se debe solicitar el cambio de dirección de despac

In [ ]:
get_answer('¿Puedo solicitar materiales telefónicamente?', 3, True)


    --Top 1------------------------------
      pregunta similar    : ¿Puedo solicitar materiales telefónicamente?
      respuesta asosciada : Las empresas que poseen una cuenta con Chilexpress pueden solicitar embalajes y materiales a través de nuestro Portal Empresas en la sección Empresas de www.chilexpress.cl
      valor similitud     : 0.5392841696739197
    --------------------------------------
    

    --Top 2------------------------------
      pregunta similar    : ¿Existen restricciones en los tiempos de entrega para un envío sobredimensionado?
      respuesta asosciada : Sí, para los envíos sobredimensionados no están disponibles los siguientes servicios: entrega en 2 horas, ultra rápido ni AM/PM. A su vez, debido al peso y a las dimensiones de este tipo de encomiendas, se sumará un día hábil en la promesa de entrega contratada.
      valor similitud     : 0.3486579656600952
    --------------------------------------
    

    --Top 3------------------------------
      p

In [ ]:
get_answer('¿Qué es un envío sobredimensionado?', 3, True)


    --Top 1------------------------------
      pregunta similar    : ¿Qué es un envío sobredimensionado?
      respuesta asosciada : Es toda encomienda que no cumpla el estándar establecido por Chilexpress, ya que posee una o más de las siguientes características: tiene un embalaje irregular, pesa más de 50 kilos y sobrepasa alguna de estas tres medidas: 1,2 metros (largo) x 0,8 metros (ancho) x 0,8 metros (alto). Puedes revisar las sucursales disponibles para estos envíos en la sección Sucursales en www.chilexpress.cl
      valor similitud     : 0.6297675967216492
    --------------------------------------
    

    --Top 2------------------------------
      pregunta similar    : ¿Cuánto cuesta un envío sobredimensionado?
      respuesta asosciada : El costo del servicio corresponderá al valor propio del envío, más un cobro adicional fijo de $3.500 + IVA dado que requiere un manejo distinto en nuestro Centro de Distribución.
      valor similitud     : 0.6245648264884949
    ------

In [ ]:
get_answer('¿Puedo cobrar un giro en cualquier sucursal de Chilexpress?', 3, True)


    --Top 1------------------------------
      pregunta similar    : ¿Puedo cobrar un giro en cualquier sucursal de Chilexpress?
      respuesta asosciada : No, sólo en las sucursales Chilexpress con sistema Western Union. Si eres el destinatario, deberás presentar tu documento de identificación junto al número de MTCN. Puedes revisar las sucursales que tienen el servicio giros disponible en www.chilexpress.cl
      valor similitud     : 0.6743813157081604
    --------------------------------------
    

    --Top 2------------------------------
      pregunta similar    : ¿Puedo ir a retirar un documento a cualquier oficina de Chilexpress?
      respuesta asosciada : No. Debes retirar tu documento en la sucursal donde se encuentra disponible para retiro. Ésta es aquella que fue seleccionada cuando se contrató el servicio.
      valor similitud     : 0.600417971611023
    --------------------------------------
    

    --Top 3------------------------------
      pregunta similar    

## 2.4 Reporte el resultado con textos de prueba.

Se indican los siguientes ejemplos utilizando el modelo Doc2Vec (modificación de Word2Vec).

In [ ]:
text = 'puedo cambiar la dirección de envío?'
get_answer(text)

No es posible. Sin embargo, si un envío no puede ser entregado en la dirección de destino original, el destinatario podrá solicitar que sea derivado a una sucursal Chilexpress ubicada en la misma comuna del despacho original.


In [ ]:
text = 'como hago seguimiento?'
get_answer(text)

No es posible hacer seguimiento de un envío sin la Orden de Transporte. Puedes obtenerlo iniciando tu sesión en www.chilexpress.cl o revisar en tu App el Histórico de envíos.


In [ ]:
text = 'quiero retirar una encomienda a otra sucursal'
get_answer(text)

No, sólo en las sucursales Chilexpress con sistema Western Union. Si eres el destinatario, deberás presentar tu documento de identificación junto al número de MTCN. Puedes revisar las sucursales que tienen el servicio giros disponible en www.chilexpress.cl


In [ ]:
text = 'puede ir otra persona a buscar mi pedido'
get_answer(text)

Sí, Chilexpress recibe tus envíos para hacerlos llegar a los distintos lugares del mundo. Puedes enviar tanto documentos como muestras al extranjero; esta última corresponde a todo aquello que no es considerado como documento y que se encuentre en el rango de 2,5 kilos a 50 kilos. Toda muestra debe venir valorada comercialmente por el remitente mediante el uso de una “Factura Comercial” expresada en dólares. Revisa más detalles en la Guía de Servicio.


In [ ]:
text = 'como retiro mi envío por pagar'
get_answer(text)

Una vez solicitado el retiro en el Portal Empresas, debes ir dentro del mismo sitio a la sección OT Digital. Para conocer cómo utilizarlo bajo el manual disponible en la sección correspondiente.


# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

In [1]:
!pip install transformers
from transformers import pipeline, AutoTokenizer, Trainer, TrainingArguments, AutoModel, AutoModelForSequenceClassification

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

### 3.1.1 Clasificador

En la presente sección se entrena un clasificador con una modificación de BERT en español: BETO. Se han utilizado los pesos preentrenados de BETO desarrollados por el DCC para el tokenizador. 

ref: https://benjad.github.io/2020/08/04/clasificador-sentimiento-BERT/

In [2]:
import torch
from transformers import  BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import torch.optim
import numpy as np
import pandas as pd
import time
import datetime
import random
from sklearn.metrics import confusion_matrix


In [9]:
# Select cpu or cuda
run_on = 'cpu'
device = torch.device(run_on)

# Load the dataset into a pandas dataframe.
messages['class'] = messages['Clase'].map({'información':0, 'nombre':1, 'saludo':2, 'despedida':3,})
clase = messages['class']
mensaje = messages['Mensaje']

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(mensaje, 
clase, stratify=clase, test_size=0.2, random_state=42)

# Report datasets lenghts
print('Training set length : {}'.format(len(X_train)))
print('Validation set length : {}'.format(len(X_val)))

Training set length : 147
Validation set length : 37


In [10]:
# < 2 min
# Tokenization  
%cd /content
!pip install urllib3==1.25.10
!rm -rf transformers
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770

!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz -O pytorch_weights.tar.gz
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt -O vocab.txt
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json -O config.json
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=True)

def preprocessing(dataset):
    input_ids = []
    attention_mask = []
    for doc in dataset:
        encoded_doc = tokenizer.encode_plus(doc,
                   add_special_tokens=True, max_length=115,
                   truncation=True,pad_to_max_length=True)
        input_ids.append(encoded_doc['input_ids'])
        attention_mask.append(encoded_doc['attention_mask'])
    return (torch.tensor(input_ids),
           torch.tensor(attention_mask))

# Apply preprocessing to dataset
X_train_inputs, X_train_masks = preprocessing(X_train)
X_val_inputs, X_val_masks = preprocessing(X_val)

# Report max n° tokens in a sentence
max_len = max([torch.sum(sen) for sen in X_train_masks])
print('Max n°tokens in a sentence: {0}'.format(max_len))

/content
Cloning into 'transformers'...
remote: Enumerating objects: 94773, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 94773 (delta 42), reused 61 (delta 41), pack-reused 94697
Receiving objects: 100% (94773/94773), 80.08 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (68803/68803), done.
Note: checking out 'a3085020ed0d81d4903c50967687192e3101e770'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a3085020e Added repetition penalty to PPLM example (#2436)
--2021-12-31 00:28:54--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.g

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Para el entrenamiento se utilizan batches de tamaño 2. El modelo corresponde a 'BertForSequenceClassification' y el optimizador es de tipo 'AdamW'.  

In [17]:
# Data loaders
batch_size = 2

y_train_labels = torch.tensor(y_train.values)
y_val_labels = torch.tensor(y_val.values)


def dataloader(x_inputs, x_masks, y_labels):
    data = TensorDataset(x_inputs, x_masks, y_labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler,
                 batch_size=batch_size,
                 num_workers=0)
    return dataloader

train_dataloader = dataloader(X_train_inputs, X_train_masks,
                   y_train_labels)
val_dataloader = dataloader(X_val_inputs, X_val_masks, 
                 y_val_labels)

In [18]:
# set random seed
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed_all(value)
set_seed(42)

# Create model and optimizer
model = BertForSequenceClassification.from_pretrained(
        "pytorch/", num_labels=4, output_attentions=False,
         output_hidden_states=False)

optimizer = AdamW(model.parameters(),
                  lr = 4e-5,
                  eps = 1e-6
                  )

if run_on == 'cuda':
    model.cuda()

# Define number of epochs
epochs = 3

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
            num_warmup_steps = 0, 
            num_training_steps = total_steps)

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pytorch/

In [19]:
#fuction to format time
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#function to compute accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Desarrollo del entrenamiento. 

In [20]:
# ~8 min
#function to train the model
def training(n_epochs, training_dataloader,
             validation_dataloader):
    # ========================================
    #               Training 
    # ========================================
    print('======= Training =======')
    for epoch_i in range(0,n_epochs):
        # Perform one full pass over the training set
        print("")
        print('======= Epoch {:} / {:} ======='.format(
             epoch_i + 1, epochs))
        # Measure how long the training epoch takes.
        t0 = time.time()
        # Reset the total loss for this epoch.
        total_loss = 0
        # Put the model into training mode.
        model.train()
        # For each batch of training data
        for step, batch in enumerate(training_dataloader):
            batch_loss = 0
            # Unpack this training batch from dataloader
            #   [0]: input ids, [1]: attention masks, 
            #   [2]: labels
            b_input_ids,b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)

            # Clear any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass 
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)

            # pull loss value out of the output tuple
            loss = outputs[0]
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass 
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                            1.0)

            # Update parameters
            # ¿take a step using the computed gradient
            optimizer.step()
            scheduler.step()

            print('batch loss: {0} | avg loss: {1}'.format(
                  batch_loss, total_loss/(step+1)))
        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)


        print("")
        print("  Average training loss: {0:.2f}".
             format(avg_train_loss))
        print("  Training epoch took: {:}".format(
              format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, 
        # measure accuracy on the validation set.

        print("")
        print("======= Validation =======")

        t0 = time.time()

        # Put the model in evaluation mode
        model.eval()

        # Tracking variables
        eval_loss, eval_accuracy = 0, 0
        all_logits = []
        all_labels = []
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Add batch to device
            # Unpack this training batch from our dataloader.
            #   [0]: input ids, [1]: attention masks,
            #   [2]: labels
            b_input_ids, b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)


            # Model will not to compute gradients
            with torch.no_grad():
                # Forward pass 
                # This will return the logits 
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)

            # The "logits" are the output values 
            # prior to applying an activation function 
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            b_labels = b_labels.to('cpu').numpy()

            # Save batch logits and labels 
            # We will use thoses in the confusion matrix
            predict_labels = np.argmax(
                             logits, axis=1).flatten()
            all_logits.extend(predict_labels.tolist())
            all_labels.extend(b_labels.tolist())

            # Calculate the accuracy for this batch
            tmp_eval_accuracy = flat_accuracy(
                                logits, b_labels)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

        # Report the final accuracy for this validation run.
        print("  Accuracy: {0:.2f}".
              format(eval_accuracy / (step+1)))
        print("  Validation took: {:}".format(
             format_time(time.time() - t0)))

    #print the confusion matrix"
    conf = confusion_matrix(
           all_labels, all_logits, normalize='true')
    print(conf)
    print("")
    print("Training complete")

#call function to train the model
training(epochs, train_dataloader, val_dataloader)

======= Training =======

======= Epoch 1 / 3 =======
batch loss: 1.4193904399871826 | avg loss: 1.4193904399871826
batch loss: 1.2601104974746704 | avg loss: 1.3397504687309265
batch loss: 1.452715277671814 | avg loss: 1.3774054050445557
batch loss: 1.019685983657837 | avg loss: 1.287975549697876
batch loss: 1.2431834936141968 | avg loss: 1.2790171384811402
batch loss: 1.7542508840560913 | avg loss: 1.358222762743632
batch loss: 0.9863739013671875 | avg loss: 1.3051014968327113
batch loss: 0.9666337966918945 | avg loss: 1.2627930343151093
batch loss: 0.8847402334213257 | avg loss: 1.2207871675491333
batch loss: 0.4400166869163513 | avg loss: 1.142710119485855
batch loss: 1.353920578956604 | avg loss: 1.1619110703468323
batch loss: 1.319498896598816 | avg loss: 1.1750433892011642
batch loss: 1.8486824035644531 | avg loss: 1.2268617749214172
batch loss: 1.8884851932525635 | avg loss: 1.2741205905164992
batch loss: 1.2667768001556396 | avg loss: 1.2736310044924417
batch loss: 0.448590099

### 3.1.2 Extraccion de respuestas para el caso "informacion"

* Se copia la sección Q&A de la clase práctica de transformers. Es decir, el modelo de **huggingface**:
[distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es](https://https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es) 
con pipeline: **question-answering**.

In [21]:
# Transformer QA
beto_qa = pipeline('question-answering', 
        model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False})
    )


Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

* Para el texto del contexto, simplemente, se concatenan todas las respuestas de testQA.csv y se genera un solo string:

In [31]:
context_text='. '.join(qa.Respuesta.tolist())

* La función **beto_get_answer()** se utiliza para entregar una respuesta mas amigable al usuario en caso de estar utilizando un chat bot real y que no encuentre una respuesta adecuada.

In [34]:
def beto_get_answer(question):
  respuesta = beto_qa({'question': question, 'context': context_text})
  if respuesta['score'] < 0.01:
    return "Perdón, pero no tengo respuesta para esa pregunta."
  else:
    return respuesta['answer']

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

### 3.2.1

* Para la sección 3.1.1 (BertForSequenceClassification) se entrenó el modelo con los pesos preentrenados de BETO (BERT para español) en el tokenizador. La métrica utilizada para optimizar el clasificador fue accuracy. Se obtuvo un accuracy de 0.97.- En general, identifica todos los textos correspondientes a 'información', 'nombre' y 'despedida', y obtiene 0.9 de accuracy para la clase 'saludo'. 


### 3.2.2

* Para la seccion 3.1.2 (Q&A), el modelo de **huggingface**:\
[distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es](https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es) también está basado en BETO.

* La versión de BETO que utiliza es [dccuchile/bert-base-spanish-wwm-cased](https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased)
* Sobre esto se le aplica *fine tuning* utilizando el dataset [SQuAD 2.0 en Español](https://github.com/ccasimiro88/TranslateAlignRetrieve/blob/master/SQuAD-es-v2.0/) (consistente en preguntas y respuestas) para re-entrenar.
*   Aplicada *Distillation* que otorga mejora comprobada en accuracy. Y mejora reportada en eficiencia al entrenar. 

## 3.3 Reporte el resultado con los textos de prueba.

In [36]:
print('¿Sólo las empresas pueden tener una cuenta con Chilexpress?')
print(beto_get_answer('¿Sólo las empresas pueden tener una cuenta con Chilexpress?')+"\n")
print('¿puedo retirar un giro de dinero?')
print(beto_get_answer('¿puedo retirar un giro de dinero?')+"\n")
print('Puede otra persona retirar mi pedido')
print(beto_get_answer('Puede otra persona retirar mi pedido')+"\n")
print('¿Qué es la Orden de Transporte (OT)?')
print(beto_get_answer('¿Qué es la Orden de Transporte (OT)?')+"\n")
print('¿Qué es un envío sobredimensionado?')
print(beto_get_answer('¿Qué es un envío sobredimensionado?')+"\n")
print('¿Quién ganará la próxima elección?')
print(beto_get_answer('¿Quién ganará la próxima elección?')+"\n")

¿Sólo las empresas pueden tener una cuenta con Chilexpress?
Sí, sólo las empresas pueden obtener una cuenta con Chilexpress.

¿puedo retirar un giro de dinero?
Una tercera persona puede retirar un envío por pagar

Puede otra persona retirar mi pedido
Debes retirar tu encomienda en la sucursal donde se encuentra disponible para retiro.

¿Qué es la Orden de Transporte (OT)?
el número asociado al envío.

¿Qué es un envío sobredimensionado?
Perdón, pero no tengo respuesta para esa pregunta.

¿Quién ganará la próxima elección?
Perdón, pero no tengo respuesta para esa pregunta.



# Conclusión

El desarrollo de esta tarea permite concluir que los métodos anteriores a transformers, en los que se utilizan embeddings como Word2Vec (desarrollado en esta tarea) o Elmo, tiene una mayor dificultad lograr entrenar un modelo que pueda identificar el tipo de preguntas o cuáles son las palabras claves de ellas. Se necesita una gran cantidad de ejemplos clasificados para lograr buenos resultados. 

En el caso de las redes tipo transformers, se utiliza un modelo preentrenado como BERT o BETO (BERT en español) como tokenizador. De esta forma, se utiliza un modelo entrenado en *todo* el idioma con lo que se pueden identificar las clases de las frases con una mayor precisión, con la misma cantidad de datos. 